In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from AnsatzLacs import *
from qiskit_algorithms.optimizers import SPSA
from PurificationUtil import *

In [ ]:
# implementation of the slack variable method
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
ref_size = 2 # size of the reference state (for purification)
total_size = sys_size + ref_size

swap_test_unitary = get_swap_unitary(sys_size)
problem_seed = 1234

In [ ]:
# calculate the trace distance
def trace_distance(rho, sigma):
    eig, _ = np.linalg.eig(rho - sigma)
    eig = np.abs(eig)
    return 0.5 * np.sum(eig)

In [ ]:
rho_ansatz = General(sys_size, sys_size, 2)
sigma_ansatz = General(sys_size, sys_size, 2)

np.random.seed(problem_seed)
rho_param = np.random.random(rho_ansatz.get_parameter_num())
sigma_param = np.random.random(sigma_ansatz.get_parameter_num())

rho_ansatz.set_params(rho_param)
sigma_ansatz.set_params(sigma_param)

rho = rho_ansatz.get_density_matrix()
sigma = sigma_ansatz.get_density_matrix()

expected = trace_distance(rho, sigma)
print("Exact trace distance:", expected)

In [ ]:
def get_trace(ansatz1, ansatz2, shots):
    return get_trace_helper(swap_test_unitary, ansatz1, ansatz2, shots)

In [ ]:
# precomputed terms
rho_squared = get_trace(rho_ansatz, rho_ansatz, None)
sigma_squared = get_trace(sigma_ansatz, sigma_ansatz, None)
rho_sigma = get_trace(rho_ansatz, sigma_ansatz, None)

In [ ]:
# customized SPSA optimizer
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation * delta)
    minus = fun(params - perturbation * delta)
    grad = (plus - minus) / (2 * perturbation * delta)

    norm = np.linalg.norm(grad)
    if norm > 1:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .99, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Dual Optimization

In [ ]:
def dual_cost_function_full(params):
    """
    Returns the value of the cost function
    params: [parameters for 1, parameters for 2, lmbda, mu]
    """
    # get new parameters
    param1 = params[:dual_num_params]
    param2 = params[dual_num_params:2 * dual_num_params]
    ansatz1.set_params(param1) # omega
    ansatz2.set_params(param2) # tau
    lmbda = (params[-2])**2
    mu = (params[-1])**2
    
    # evaluate objective function
    omega_squared = get_trace(ansatz1, ansatz1, num_shots)
    tau_squared = get_trace(ansatz2, ansatz2, num_shots)
    termA = (lmbda**2) * omega_squared + rho_squared + sigma_squared + (mu**2) * tau_squared
    
    omega_rho = get_trace(rho_ansatz, ansatz1, num_shots)
    omega_sigma = get_trace(sigma_ansatz, ansatz1, num_shots)
    omega_tau = get_trace(ansatz1, ansatz2, num_shots)
    termB = -lmbda * omega_rho + lmbda * omega_sigma - lmbda * mu * omega_tau
    
    rho_tau = get_trace(rho_ansatz, ansatz2, num_shots)
    sigma_tau = get_trace(sigma_ansatz, ansatz2, num_shots)
    termC = -rho_sigma + mu * rho_tau - mu * sigma_tau
    
    obj = lmbda
    penalty = termA + 2 * (termB + termC)
    
    cost = obj + c * penalty
    
    return cost, obj, penalty

def dual_cost_function(params):
    cost, _, _ = dual_cost_function_full(params)
    return cost

In [ ]:
print_every = 100
def dual_callback(x, iteration):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global lr
    global lr_wait
    
    fx, obj, penal = dual_cost_function_full(x)
    dual_iterations.append(iteration)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objs.append(obj)
    
    if (iteration % print_every == 0):
        lr_wait += 1
        if lr_wait >= 5:
            m, _ = np.polyfit(dual_iterations[-500:], dual_loss[-500:], 1)
            if m > 0:
                lr = max(lr / 2,lr_min)
                lr_wait = 0
        
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12

max_iter = 30 * 1000

lr_max = 1e-1
lr_min = 1e-3
perturbation = 1e-4

dual_num_layers = 3
dual_num_params = get_parameter_number('general', sys_size, ref_size, dual_num_layers)

In [ ]:
iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []
record = 0
num_train = 10

for i in range(num_train):
    print(f"\nExpected Value: {round(expected,5)}")
    lr = lr_max
    lr_wait = 0
    lmbda0 = 1.
    mu0 = 1.
    c = 100
    
    np.random.seed(i)
    ansatz1 = create_ansatz("general", sys_size, ref_size, dual_num_layers)
    ansatz2 = create_ansatz("general", sys_size, ref_size, dual_num_layers)

    init_params = [*ansatz1.params, *ansatz2.params, np.sqrt(lmbda0), np.sqrt(mu0)]
    init_cost, init_obj, init_penal = dual_cost_function_full(init_params)
    
    dual_iterations = [0]
    dual_loss = [init_cost]
    dual_penals = [init_penal]
    dual_objs = [init_obj]
    
    print(f"\nIteration: {0}", "Loss: {:.4g}".format(init_cost), "Objective: {:.3g}".format(init_obj),
              "Penalty: {:.3g}".format(init_penal), "C: {:.2g}".format(c), sep = '\t')
    result = gradient_descent(dual_cost_function, init_params, get_gradient, max_iter, callback=dual_callback)
    
    iterations_dual_outer.append(dual_iterations)
    costs_dual_outer.append(dual_loss)
    penalties_dual_outer.append(dual_penals)
    objs_dual_outer.append(dual_objs)
    
    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"TD_Purification_dual_data"
        problem_name = "TD"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objs, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}") 

# Primal Optimization

In [ ]:
def primal_cost_function_full(params):
    """
    Returns the value of the cost function
    params: [parameters for 1, parameters for 2, lmbda, mu]
    """
    # get new parameters
    param1 = params[:primal_num_params]
    param2 = params[primal_num_params:2 * primal_num_params]
    ansatz1.set_params(param1)
    ansatz2.set_params(param2)
    lmbda = (params[-2]) ** 2
    mu = (params[-1]) ** 2
    
    # evaluate objective function
    omega_squared = get_trace(ansatz1, ansatz1, num_shots)
    tau_squared = get_trace(ansatz2, ansatz2, num_shots)
    
    omega_rho = get_trace(rho_ansatz, ansatz1, num_shots)
    omega_sigma = get_trace(sigma_ansatz, ansatz1, num_shots)
    omega_tau = get_trace(ansatz1, ansatz2, num_shots)
    
    obj = lmbda * (omega_rho + -omega_sigma)
    
    penaltya = 2 ** sys_size + (lmbda ** 2) * omega_squared + (mu ** 2) * tau_squared
    penaltyb = lmbda * mu * omega_tau - lmbda - mu
    
    penalty = penaltya + 2 * penaltyb
    
    cost = obj - c * penalty
    return cost, obj, penalty

def primal_cost_function(params):
    cost, _, _ = primal_cost_function_full(params)
    return -cost

In [ ]:
print_every = 100
def primal_callback(x, iteration):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global lr
    global lr_wait
    
    fx, obj, penal = primal_cost_function_full(x)
    primal_iterations.append(iteration)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objs.append(obj)
    
    if (iteration % print_every == 0):
        lr_wait += 1
        if lr_wait >= 5:
            m, _ = np.polyfit(primal_iterations[-500:], primal_loss[-500:], 1)
            if m < 0:
                lr = max(lr / 2,lr_min)
                lr_wait = 0
        
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12

max_iter = 30 * 1000

lr_max = 1e-1
lr_min = 1e-3
perturbation = 1e-4

primal_num_layers = 3
primal_num_params = get_parameter_number('general', sys_size, ref_size, primal_num_layers)

In [ ]:
iterations_primal_outer = []
costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []
record = 0
num_train = 10

for i in range(num_train):
    print(f"\nExpected Value: {round(expected,5)}")
    lr = lr_max
    lr_wait = 0
    lmbda0 = 1.
    mu0 = 1.
    c = 10
    
    np.random.seed(i)
    ansatz1 = create_ansatz("general", sys_size, ref_size, primal_num_layers)
    ansatz2 = create_ansatz("general", sys_size, ref_size, primal_num_layers)

    init_params = [*ansatz1.params, *ansatz2.params, np.sqrt(lmbda0), np.sqrt(mu0)]
    init_cost, init_obj, init_penal = primal_cost_function_full(init_params)
    
    primal_iterations = [0]
    primal_loss = [init_cost]
    primal_penals = [init_penal]
    primal_objs = [init_obj]
    
    print(f"\nIteration: {0}", "Loss: {:.4g}".format(init_cost), "Objective: {:.3g}".format(init_obj),
              "Penalty: {:.3g}".format(init_penal), "C: {:.2g}".format(c), sep = '\t')
    result = gradient_descent(primal_cost_function, init_params, get_gradient, max_iter, callback=primal_callback)
    
    iterations_primal_outer.append(primal_iterations)
    costs_primal_outer.append(primal_loss)
    penalties_primal_outer.append(primal_penals)
    objs_primal_outer.append(primal_objs)
    
    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"TD_Purification_primal_data"
        problem_name = "TD"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objs, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}") 

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 100
window_size_dual = 100

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(-8, 3)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()